#### Description of the file

##### 0 - Setup

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import sklearn
import statsmodels as sm
import numba
from pathlib import Path

In [2]:
DIR_DATA = Path("..") / "data"
DIR_RESULTS = Path("..") / "results"
DIR_STOCK_DATA = DIR_DATA / "stock_data"

PATH_META = DIR_DATA / "metadata_listed.csv"
PATH_META_DELISTED = DIR_DATA / "metadata_delisted.csv"

In [3]:
df_meta = pd.read_csv(PATH_META)
df_target = df_meta.copy()

In [4]:
df_meta_delisted = pd.read_csv(PATH_META_DELISTED)
df_target_delisted = df_meta_delisted.copy()

In [5]:
all_stocks = []
p = 0.1

for row in tqdm.tqdm(df_target.itertuples(index=False), total=len(df_target), desc="Loading listed stocks"):
    x = np.random.binomial(1, p)
    if x == 0:
        continue
    
    symbol = row.Symbol
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    try:
        df_stock = pd.read_csv(path_stock)
    except Exception as e:
        continue

    df_stock.index = pd.MultiIndex.from_product([[symbol], df_stock.index], names=['ticker', 'date'])

    all_stocks.append(df_stock)

for row in tqdm.tqdm(df_target_delisted.itertuples(index=False), total=len(df_target_delisted), desc="Loading delisted stocks"):
    x = np.random.binomial(1, p)
    if x == 0:
        continue
    
    symbol = row.Symbol
    path_stock = DIR_STOCK_DATA / f"{symbol}.csv"
    try:
        df_stock = pd.read_csv(path_stock)
    except Exception as e:
        continue

    df_stock.index = pd.MultiIndex.from_product([[symbol], df_stock.index], names=['ticker', 'date'])

    all_stocks.append(df_stock)

df_stocks = pd.concat(all_stocks)

Loading delisted stocks: 100%|██████████| 9720/9720 [00:27<00:00, 354.40it/s]


In [6]:
df_stocks.sample(10)

,,ticker,date,open,high,low,close,adj_close,volume
ticker,date,,,,,,,,
TBI,2334,TBI,2003-07-10,7.500000,7.500000,7.170000,7.170000,7.170000,152600.0
ATLC,3686,ATLC,2013-12-16,3.640000,3.640000,3.640000,3.640000,3.640000,0.0
LKQ,4021,LKQ,2019-09-26,31.480000,31.709999,31.290001,31.430000,31.430000,1701700.0
DECK,4654,DECK,2012-04-09,63.779999,64.519997,63.509998,64.010002,64.010002,1369500.0
TSH,16,TSH,1995-05-11,11.625000,11.625000,11.625000,11.625009,11.625009,5800.0
EF,2067,EF,1998-07-01,21.562500,21.562500,21.562500,42.324613,42.324613,34400.0
DFG,583,DFG,1999-02-25,51.687500,51.687500,51.687500,54.851059,54.851059,288500.0
SILI,5527,SILI,1994-11-18,12.500000,12.500000,12.500000,10.449765,10.449765,545.0
GUL,6920,GUL,1997-03-05,9.687500,9.687500,9.687500,164.609422,164.609422,7700.0
